# Maping the wave of spread of covid-19

In [29]:
import outbreak
import pandas as pd
import geopandas as gpd
import copy
import os
os.environ['MAPBOX_API_KEY'] = 'pk.eyJ1IjoiZmNjb2VsaG8iLCJhIjoiY2s4c293dzc3MGJodzNmcGEweTgxdGpudyJ9.UmSRs3e4EqTOte6jYWoaxg'
import pydeck
import ujson

## Loading the data

In [2]:
F = outbreak.read_flow_matrix('../csv/flowmatrix_full_mun.csv.gz', header=0) 
nodes = outbreak.read_nodes('../csv/nodesUF.csv.gz')
# nodes['incidence'] = nodes.I/nodes.tgtpop
mapa = gpd.read_file('../malha/muni_br.gpkg')
# Fetching cases from Brasil.io
cases = pd.read_csv('https://brasil.io/dataset/covid19/caso?format=csv')
nodes = pd.merge(nodes,cases[(cases.is_last)], left_on='tgt', right_on='city_ibge_code', how='left')
nodes = nodes[['tgt','tgtname','tgt_UFCOD','tgtpop','confirmed']]
nodes.fillna(0, inplace=True)
cases

,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-04-10,AC,Acrelândia,city,9,0.0,True,15256.0,1200013.0,58.99318,NaN
1,2020-04-10,AC,Bujari,city,1,0.0,True,10266.0,1200138.0,9.74089,NaN
2,2020-04-10,AC,Plácido de Castro,city,4,0.0,True,19761.0,1200385.0,20.24189,NaN
3,2020-04-10,AC,Porto Acre,city,1,0.0,True,18504.0,1200807.0,5.40424,NaN
4,2020-04-10,AC,Rio Branco,city,55,2.0,True,407319.0,1200401.0,13.50293,0.0364
...,...,...,...,...,...,...,...,...,...,...,...
10407,2020-02-27,SP,São Paulo,city,1,NaN,False,12252023.0,3550308.0,0.00816,NaN
10408,2020-02-27,SP,NaN,state,1,0.0,False,45919049.0,35.0,0.00218,NaN
10409,2020-02-26,SP,NaN,state,1,0.0,False,45919049.0,35.0,0.00218,NaN
10410,2020-02-25,SP,São Paulo,city,1,NaN,False,12252023.0,3550308.0,0.00816,NaN


Add to map only the latest count per municipality

In [3]:
mapa['casos']=0
for i, row in mapa.iterrows():
    if row.code_muni in list(cases.city_ibge_code):
        mapa.loc[mapa.code_muni==row.code_muni, 'casos'] += cases[(cases.city_ibge_code==row.code_muni)&
                                                                  (cases.is_last)].confirmed.iloc[0]

In [4]:
def get_seeds(nodes, threshold=100):
    '''
    Only consider cities with more than `threshold` cases as seeds
    '''
    seeds = copy.deepcopy(nodes)
    seeds.confirmed = [c if c>=100 else 0 for c in nodes.confirmed]
    # nodes = nodes[nodes.confirmed >=100]
    seeds.fillna(0,inplace=True)
    seeds['incidence'] = (seeds.confirmed/seeds.tgtpop)
    return seeds

In [5]:
attenuation = 1 # 1 means no attenuation
r0=2.5
seeds = get_seeds(nodes,100)
probs = outbreak.get_outbreaks(F, seeds.incidence, R0=r0, attenuate=attenuation)
probs

array([0.        , 0.        , 0.        , ..., 0.77754959, 0.11005735,
       1.        ])

In [7]:
nodes['probs'] = probs
mapap = pd.merge(mapa, nodes, left_on='code_muni', right_on='tgt')

In [40]:
DATA = ujson.loads(mapap[mapap.probs>0.5].to_json())

In [42]:

LAND_COVER = [[[-54.291, -14.139], [-54.39180562, -14.23962137], [-54.39180562, -14.23962137], [-54.291, -14.139]]]

INITIAL_VIEW_STATE = pydeck.ViewState(
  latitude=-14.23962137,
  longitude= -54.39180562,
  zoom=3,
  max_zoom=16,
  pitch=45,
  bearing=0
)

polygon = pydeck.Layer(
    'PolygonLayer',
    LAND_COVER,
    stroked=False,
    # processes the data as a flat longitude-latitude pair
    get_polygon='-',
    get_fill_color=[0, 0, 0, 20]
)

geojson = pydeck.Layer(
    'GeoJsonLayer',
    DATA,
    opacity=0.8,
    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
#     mapbox_key='pk.eyJ1IjoiZmNjb2VsaG8iLCJhIjoiY2s4c293dzc3MGJodzNmcGEweTgxdGpudyJ9.UmSRs3e4EqTOte6jYWoaxg',
    get_elevation='properties.casos',
    get_fill_color='[properties.probs *255, 255*(1-properties.probs), properties.probs * 255]',
    get_line_color=[255, 255, 255],
    pickable=True
)

r = pydeck.Deck(
    layers=[polygon,geojson],
    initial_view_state=INITIAL_VIEW_STATE)

r.to_html()

'/home/fccoelho/Mega/Papers/Covid-19-epigrass/covid-19/Modelagem Epi/pydeck5x4bpbc8.html'

In [13]:
mapap.total_bounds.reshape((2,2)).mean(axis=0)

array([-54.39180562, -14.23962137])

In [34]:
pydeck.Layer?